In [26]:
import aiida

# must execute it in the first place
aiida.load_profile()

In [27]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node
from tools import load_atoms
from ase import io
from tools import wait_for_node_finished

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [28]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
print(computer_local)
code = get_or_create_code('alamode.displace_pf',
                          computer_local,
                          '/home/max/Documents/aiida-test/alamode-aiida/alamode/displace.py')
code_displace_pf = code
code = get_or_create_code('alamode.displace_random',
                          computer_local,
                          '/home/max/Documents/aiida-test/alamode-aiida/alamode/displace.py')
code_displace_random = code

# You must setup remote code
# if you need a setup of intel fortran, 
# . ~/intel/oneapi/setvars.sh
# is necessary.
code_simulator_name = Str('alamode.pwx@mygarden1')

code_displace_random, code_displace_pf, code_simulator_name

localhost (localhost), pk: 5


(<Code: Remote code 'alamode.displace_random' on localhost, pk: 41766, uuid: 89af1d86-2610-4cd3-b551-f10aabdefb99>,
 <Code: Remote code 'alamode.displace_pf' on localhost, pk: 41765, uuid: 5f2296df-e3ef-4831-9348-445ba664b97d>,
 <Str: uuid: d8b87133-3fc2-4ee8-878b-6472f185e166 (unstored) value: alamode.pwx@mygarden1>)

In [29]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path

{'CWD': 'run27'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run27'

In [30]:
from os.path import expanduser
from tools import NodeBank
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)

In [31]:
g_cwd = nodebank.load("cwd")  # ファイル保存directory
g_superstructure = nodebank.load("superstructure")  # 使用する長周期構造StructureData
g_superstructure_file = nodebank.load(
    "superstructure_file")  # 使用する長周期構造ファイルSinglefileData
g_superstructure_filepath = nodebank.load(
    "superstructure_filepath")  # 使用する長周期構造ファイルabspath
g_file_format = nodebank.load("structure_org_format")  # lammpsの位置フォーマット
g_file_style = nodebank.load("structure_org_style")  # lammpsの位置フォーマットのオプション
g_file_format, g_file_style, g_superstructure_file

(<Str: uuid: 981a6471-8a6e-406f-9203-5ce0b51904e6 (pk: 46045) value: lammps-data>,
 <Str: uuid: 0e5b94db-39ee-4dcc-8a2b-426ae0c433a9 (pk: 46046) value: atomic>,
 <SinglefileData: uuid: 2309129c-ef63-4cdd-b421-a0eb0e06d31a (pk: 46066)>)

## for QE

In [53]:
key = "format"
g_format = nodebank.load_or_dump(key, Str("QE"))  # alamodeのファイルオプションの定義

debug, node uuid: 5cdc82c5-7e8f-40fa-9632-b3acd454b111 (unstored) value: QE
debug, load uuid: a6fe889a-13d2-4793-81a9-58e68858eff2 (pk: 46078) value: LAMMPS


In [33]:
_dir = os.path.abspath('lammps_input/Si222')
if True:
    lammps_potentials = FolderData()
    for _p in ["Si.sw"]:
        abspath = os.path.join(_dir, _p)
        print(abspath, _p)
        lammps_potentials.put_object_from_file(abspath, path=_p)
else:
    _potential = []
    for _p in ["Si.sw"]:
        _potential.append(os.path.join(_dir, _p))
    lammps_potentials = List(list=_potential)

input_file_template = SinglefileData(os.path.join(_dir, "in.sw"))

/home/max/Documents/aiida-test/alamode-aiida/example/lammps_input/Si222/Si.sw Si.sw


In [34]:
input_file_template.get_content()

'units           metal\natom_style      atomic\nboundary        p p p\n\n#read_data       tmp.lammps\nread_data       Si222.lammps\n\npair_style      sw\npair_coeff \t* * Si.sw Si\n\ndump            1 all custom 1 XFSET id xu yu zu fx fy fz \ndump_modify     1 format float "%20.15f"\nrun             0\n\n'

In [35]:
g_mag = nodebank.load_or_dump("mag", Float(0.01))

debug, node uuid: 4683e69f-8bf6-49e0-877b-d6f4a822b768 (unstored) value: 0.01
debug, load uuid: 054505e0-0a57-4cab-9195-a401a57eebb2 (pk: 46079) value: 0.01


In [36]:
g_pattern_files = nodebank.load("harmonic_pattern_folder")  # 作成したpattern files
g_pattern_files

<FolderData: uuid: f3bbf243-f2af-4d9e-a466-b0c586ec8fba (pk: 46077)>

In [37]:
g_pattern_files.list_object_names()

['harmonic.pattern_HARMONIC']

In [38]:
g_pattern_files.get_object_content('harmonic.pattern_HARMONIC')

'Basis : C\n    1:    1\n      1              1              0              0\n'

In [39]:
g_prefix = nodebank.load('prefix')
g_prefix

<Str: uuid: f9ac64da-a670-4215-bd4c-88d62c173c05 (pk: 46069) value: harmonic>

In [40]:
g_action = "random"

if g_action == "pf":

    #codename = "alamode.displace_pf@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_pf

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure_file
    builder.mag = g_mag
    builder.pattern_files = g_pattern_files

    g_displacefuture = nodebank.load("displace_pf")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump("displace_pf", g_displacefuture)

elif g_action == "random":

    g_num_disp = nodebank.load_or_dump("num_disp", Int(5))

    #codename = "alamode.displace_random@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_random

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure_file
    builder.mag = g_mag
    builder.num_disp = g_num_disp

    g_displacefuture = nodebank.load("displace_random")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump("displace_random", g_displacefuture)
        else:
            raise
else:
    raise ValueError("unknown action")


debug, node uuid: f45ad5f0-f3a3-41cc-8041-d60351d9e1a4 (unstored) value: 5
debug, load uuid: ef63dae9-8b0b-4ab2-9089-7a1369fe535a (pk: 46080) value: 5


In [41]:
g_displacefuture


<CalcJobNode: uuid: f6630df6-c23e-4835-ac11-b42f88038f2d (pk: 46083) (aiida.calculations:alamode.displace_random)>

In [42]:
g_displacefuture.outputs


<NodeLinksManager: Manager for outgoing CREATE links for node pk=46083>

In [43]:
g_displace_result = g_displacefuture.outputs.result
g_displace_result


<Dict: uuid: 938e428e-a08d-49eb-8550-e3a1ba166921 (pk: 46086)>

In [44]:
g_dispfile_folder = g_displacefuture.outputs.dispfile_folder


In [45]:
g_dispfile_folder.list_object_names()  # lammps data files


['harmonic1.lammps',
 'harmonic2.lammps',
 'harmonic3.lammps',
 'harmonic4.lammps',
 'harmonic5.lammps']

# run all the displacements

In [51]:
from aiida.orm.nodes.data.upf import get_pseudos_from_structure

def _make_upf_locally( structure: StructureData, pseudos: str, cwd: str=None) -> dict:
    upflist = get_pseudos_from_structure(structure, pseudos)
    upfdic = {}
    for upfkey, upfvalue in upflist.items():
        #print(upfvalue.get_content())
        upf_filename = upfvalue.list_object_names()[0]
        upfdic[upfkey] = upf_filename
        if cwd is not None:
            target_path = os.path.join(cwd, upf_filename)
            with open(target_path,"w") as f:
                f.write(upfvalue.get_content())
            print(target_path,"is made.")
    return upfdic

In [52]:
def make_qe_input(superstructure: StructureData, pseudos: (str,Str), QEinput = "qe.in"):
    params = {
            'calculation': 'scf',
             'tprnfor': True, 
             'tstress': True,
            'pseudo_dir': ".", 
            'ecutwfc': 3.,
            'ecutrho': 10.,
            'conv_thr': 5.e-6,
            'electron_maxstep': 100,
    }

    if isinstance(pseudos,Str):
        pseudos = pseudos.value
    pseudo = _make_upf_locally(superstructure, pseudos)
    kspacing  = 0.05
    koffset  = (1,1,1)
    with open(QEinput,"w") as f:
        write_espresso_in(f, superstructure.get_ase(), pseudopotentials =pseudo, 
                          kspacing= kspacing, koffset=koffset, 
                      **params)
    return True

In [46]:
alldisp_lammps_WorkChain = WorkflowFactory("alamode.dispall_lammps")


In [47]:
inputs = {"displace_result": g_displace_result,
          "data_folder": g_dispfile_folder,
          "input_file_template":  input_file_template,
          "code_string":  code_simulator_name,
          "cwd": g_cwd,
          "prefix": g_prefix,
          'potential_files': lammps_potentials}


In [48]:
lammps_all = nodebank.load("all_lammps_harmonic")
if lammps_all is None:
    lammps_all = submit(alldisp_lammps_WorkChain, **inputs)
    wait_for_node_finished(lammps_all, 5)
    print(lammps_all.is_finished_ok)


wait another 5 sec.
False False
False


In [49]:
lammps_all


<WorkChainNode: uuid: 3aa74d7a-7ad4-4467-866f-7a2c86b53916 (pk: 46098) (aiida.workflows:alamode.dispall_lammps)>

In [50]:
lammps_all.outputs.dfset


NotExistentAttributeError: Node<46098> does not have an output with link label 'dfset'

# extract

In [ ]:
g_cwd


In [ ]:
from aiida.plugins import DataFactory, WorkflowFactory
extractworkchain = WorkflowFactory("alamode.extract")

g_target_file = lammps_all.outputs.dfset
g_norder = nodebank.load("norder")


In [ ]:
g_target_file


In [ ]:
g_superstructure_file


In [ ]:
g_superstructure_file


In [ ]:
inputs = {'format': g_format,
          'structure_org': g_superstructure_filepath,
          # 'target_file': g_pwx_all_future.outputs.pwx_output,
          'target_file': g_target_file,
          'cwd': g_cwd,
          'prefix': g_prefix
          # "norder": g_norder
          # 'output_filename' : g_DFSET_filename
          }
print(inputs)


In [ ]:
from tools import wait_for_node_finished


In [ ]:
g_alamodeextract_future = run(extractworkchain, **inputs)


In [ ]:
g_alamodeextract_future = submit(extractworkchain, **inputs)
print(g_alamodeextract_future)
wait_for_node_finished(g_alamodeextract_future)
print(g_alamodeextract_future.is_finished_ok)


In [ ]:
nodebank.dump("harmic_extract", g_alamodeextract_future)
